# Machine Learning

### Was ist Machine Learning?

Machine Learning, im Deutschen maschinelles Lernen genannt, ist ein Teilbereich von künstlicher Intelligenz.
Ein Machine Learning Algorithmus versucht aus Beispieldaten Zusammenhänge zu finden, womit es dann eine Vorhersage über neue Daten machen kann.
Diese Algorithmen sind nicht explizit programmiert, sondern entwickeln sind anhand der Daten ständig weiter.

Für eine Vorhersage über neue Daten brauchen wir ein '**target**', also ein Ziel. In unserem Beispiel wird es das **Bundesland** von unseren Gemeinden sein.

Es gibt unterschiedliche Arten des maschinellen Lernens. Wir werden uns hier mit **überwachtem Lernen** beschäftigen, wo wir unseren Algorithmen die 'richtige Lösung' bei den Beispieldaten verraten.
Weiters unterscheidet man zwischen **Klassifikation** und **Regression**. Bei einer Klassifikation wollen wir einen kategorischen Wert schätzen, während wir bei der Regression versuchen, so nahe wie möglich auf den richtigen numerischen Wert zu kommen.

Da wir die Bundesländer unserer Gemeinden vorhersagen wollen und überwachtes Lernen benutzen, müssen wir 'Bundesland' zu unserem Datensatz hinzufügen. Die erste Ziffer der Gemeindekennzahl entspricht dem Bundesland. Diese Ziffern repräsentieren nicht numerische, sondern kategorische Werte, weil jede Zahl für eines der neun Bundesländer steht. Wir werden die Werte auf Text umändern, damit klar ist, dass es sich hier um Klassifikation handelt.

In [ ]:
# Import der Pandas Library
import pandas as pd

# Import des zuvor gespeicherten Datensatzes
gem_daten_sauber = pd.read_csv('gem_daten_sauber.csv', index_col=0)

# Visueller Check
gem_daten_sauber.head()

### Bundesland hinzufügen

In [ ]:
# Erste Ziffer aus 'Gemeindekennziffer' nehmen und als Bundesland codieren
gem_daten_sauber['Bundesland'] = gem_daten_sauber['Gemeindekennziffer'].astype(str).str[0]

# Datentyp von Bundesland ist derzeit als 'Objekt' gespeichert. Wir wollen es als ganze Zahl (int) haben:
gem_daten_sauber['Bundesland'] = gem_daten_sauber['Bundesland'].astype(int)

# Überblick
gem_daten_sauber.head()

In [ ]:
# Noch ein letztes Mal überprüfen, ob unsere Datentypen stimmen:
gem_daten_sauber.dtypes

In [ ]:
# Wir wollen nun die Ziffern in Namen umwandeln:
d_bundesland = {1:'Burgenland', 2:'Kärnten', 3:'Niederösterreich', 4:'Oberösterreich',
                5:'Salzburg', 6:'Steiermark', 7:'Tirol', 8:'Vorarlberg', 9:'Wien'}
gem_daten_sauber['Bundesland'].replace(d_bundesland, inplace=True)

# Überblick, um zu schauen, ob es funktioniert hat
gem_daten_sauber.head()

### Urbanisierungsgrad hinzufügen

Wir wollen zusätzlich noch den Urbanisierungsgrad von einer Gemeinde als Spalte hinzufügen. Wenn du willst, kannst du später probieren, den Urbanisierungsgrad als Ziel vom Machine Learning zu verwenden.

Quelle: [Statistik Austria](https://www.statistik.at/web_de/klassifikationen/regionale_gliederungen/stadt_land/index.html)

Unser neuer Datensatz besteht aus 3 den Spalten *GKZ*, *CODE* und *TXT.

* Über die Spalte *GKZ* können wir diesen Datensatz mit unseren sauberen Gemeindedaten (via *Gemeindekennziffer*) verbinden.
* Die Spalte *TXT*, wo die Kodierung in Textform gespeichert ist, werden wir nicht brauchen,
* weil wir die numerische Kodierung *CODE* in Text umwandeln werden.

In [ ]:
# Datensatz hinzufügen:
urbanisierungsgrad = pd.read_csv('gemeinden_urbanisierungsgrad.csv', sep=';')

# Übersicht:
urbanisierungsgrad.head()

**Achtung:** Die untere Codezelle nicht mehrmals hintereinander ausführen!

In [ ]:
# Zusammenführen der beiden Datensätze über die Gemeindekennziffern

gem_daten_sauber = gem_daten_sauber.join(urbanisierungsgrad.set_index('GKZ'), on='Gemeindekennziffer')

In [ ]:
# Visuelle Überprüfung des neuen, kombinierten Datensatzes:
gem_daten_sauber.head()

***Achtung:*** Bei manchen Codezellen muss man aufpassen, dass man sie nicht mehrmals ausführt. In der folgenden Codezelle werden nämlich Spalten aus dem Datensatz entfernt. Führt man diese Zelle nochmal aus, kommt eine Fehlermeldung, dass die Spalten, die man versucht hat zu entfernen, nicht gefunden wurden. Das passiert, weil diese Spalten schon bei der ersten Ausführung von der Codezelle entfernt wurden und nun nicht mehr existieren. Falls du auf so ein Problem stoßt, kannst du einfach alle Codezellen von oben neu ausführen.

In [ ]:
# Entfernung von Spalten die wir (fürs Machine-Learning) 
# nicht mehr brauchen:
gem_daten_sauber.drop(columns=['PLZ des Gem.Amtes', 'weitere Postleitzahlen', 'TXT'], inplace=True)

# Die Spalte 'CODE' nennen wir nun 'Urbanisierungsgrad'
gem_daten_sauber.rename(columns= {'CODE' : 'Urbanisierungsgrad'}, inplace=True)

# Leserfreundliche Umbenennung der Urbanisierungsgrade:
# 1 bedeutet 'Hoch', 2 'Mittel' und 3 ist 'Niedrig'
d_urbanisierungsgrad = {1:'Hoch', 2:'Mittel', 3:'Niedrig'}
gem_daten_sauber['Urbanisierungsgrad'].replace(d_urbanisierungsgrad, inplace=True)

# Übersicht:
gem_daten_sauber.head()

Wir wollen unserem Machine Learning Algorithmus also Daten geben, die er noch nicht kennt, damit er auf diesen unbekannten Daten das Bundesland vorhersagt.

Dafür wollen wir einen kleinen Teil unserer Daten (wir entscheiden uns hier für 2%) raus nehmen und zur Seite stellen, damit dieser am Ende für Schätzungen benutzt werden kann.

In [ ]:
# 2% der Daten ausschließen damit man sie nur zum Schätzen benutzen kann:
ml_datensatz = gem_daten_sauber.sample(frac=0.98, random_state=101)
ungesehene_daten = gem_daten_sauber.drop(ml_datensatz.index)

ml_datensatz.reset_index(drop=True, inplace=True)
ungesehene_daten.reset_index(drop=True, inplace=True)

# Neue Größe unserer Daten: (Anzahl an Zeilen, Anzahl der Spalten)
print('Daten für die Modellierung: ' + str(ml_datensatz.shape))
print('Ungesehene Daten für Schätzungen: ' + str(ungesehene_daten.shape))

In [ ]:
# Überblick über die Daten, die wir für Machine Learning verwenden:
ml_datensatz.head()

In [ ]:
# Überblick über die Daten, die wir zum Schätzen verwenden (42 insgesamt):
ungesehene_daten.head()

# Setup

Als ersten Schritt müssen wir '**setup**' ausführen. Gehen wir die wichtigsten Parameter durch: 
* **data**: Wir müssen den Datensatz, den wir verwenden wollen, angeben.
* **target**: Die Spalte, welches wir als Ziel ausgesucht haben und vorhersagen wollen.
* **ignore_features**: Spalten die ignoriert werden sollen. Wir entfernen 'Gemeindename', weil es uns keine Information über das Bundesland gibt und die Algorithmen potenziell schlechter werden. Zusätzlich ignorieren wir 'Gemeindekennziffer' und 'Gemeindecode', weil diese sofort alle Bundesländer verraten würden. 
* **train_size**: Wir unterscheiden zwischen **Trainingsdaten** und **Testdaten**. Trainingsdaten sind die Beispieldaten, die unsere Algorithmen benutzen um zu lernen, während man danach auf den Testdaten die Genauigkeit von den Algorithmen evaluiert. Wir haben uns hier für 70% Trainingsdaten und 30% Testdaten entschieden.

Wenn man auf **3: Label Encoded** schaut, sieht man, dass die verschiedenen Bundesländer automatisch als Zahlen kodiert wurden.

***Hinweis: Manche der kommenden Codezellen können bis zu einigen Minuten dauern!***

In [ ]:
# Für Machine Learning benutzen wir PyCaret als Tool und benötigen die Algorithmen für Klassifikation

from pycaret.classification import *
exp = setup(data=ml_datensatz, target='Bundesland', 
    ignore_features=['Gemeindekennziffer', 'Gemeindename', 'Gemeindecode'], 
    silent=True, session_id=101, train_size=0.70)

# Modelltest

Als zweiten Schritt wollen wir nun die verschiedenen Machine Learning Algorithmen miteinander vergleichen (**compare_models**). Es werden ein paar von ihnen aufgrund der langen Rechenzeit exkludiert.

Die Anzahl an **folds** muss bestimmt werden: 'fold=3' bedeutet, dass wir 3-mal 70% (siehe: setup) unserer Daten als Trainingsdaten verwenden. Dabei werden die Trainingsdaten jedes Mal zufällig ausgesucht und sind daher unterschiedlich.
Der Grund, wieso wir folds verwenden, ist damit wir ein sogenanntes **overfitting** vermeiden. Overfitting bedeutet, dass unser Algorithmus die Trainingsdaten zu genau gelernt hat, wodurch Probleme entstehen können, wie zum Beispiel dass er neue Daten komplett falsch schätzt, wenn sie sich zu sehr von den Trainingsdaten unterscheiden.

***Hinweis: Die Ausführung dauert in der Regel 30-60 Sekunden, kann aber bis zu 2 Minuten brauchen.***

In [ ]:
# Wir vergleichen die ML-Algorithmen miteinander:
best = compare_models(fold=3, exclude=['lightgbm'])

Die gelben Markierungen in der Tabelle heben die besten Werte hervor. In diesem Beispiel interessiert uns **Accuracy**, die Genauigkeit vom Algorithmus, am meisten.

Wir sehen, dass das *Gradient Boosting* Modell eine sehr hohe Genauigkeit *und* eine niedrige Laufzeit (**TT (Sec)**) hat und wollen den als unser Modell nehmen. Dafür brauchen wir den Befehl *create_model*.

***Hinweis: Die Ausführung dauert in der Regel 30-60 Sekunden, kann aber bis zu 2 Minuten brauchen.***

In [ ]:
# Wähle Gradient Boosting Classifier als Modell, mit 3 folds
selected_model = create_model('gbc', fold=3)

Knapp unter 60% an Genauigkeit wirkt auf dem ersten Blick nach relativ wenig, aber wenn man bedenkt, dass bei jeder Entscheidung unser Algorithmus zwischen 9 verschiedenen Bundesländern eines auswählen muss, ist es doch ganz faszinierend.

Mit **tune_model** können wir einige der Ergebnisse weiter verbessern.

Falls keine Verbesserungen hervorliegen, kann das daran liegen, dass es intern versucht overfitting zu vermeiden.

***Hinweis: Beim Ausführen dieser Zelle stockt der "Processing" Balken regelmäßig, aber die Berechnung läuft weiter! Die Ausführung kann bis zu 2 Minuten dauern.***

In [ ]:
# Mit tune_model können wir unser ausgesuchtes Modell
# (Random Forest Classifier) weiter verbessern.

tuned_model = tune_model(selected_model, fold=3)

## Auswertung der Vorhersagequalität

Es existieren viele Visualisierungen, die einem helfen können, die Ergebnisse vom Algorithmus besser zu verstehen. Wir fokussieren uns hier auf drei davon.

### Confusion Matrix

Hier siehst du die Anzahl der Daten, die richtig bzw. falsch klassifiziert wurden. Auf der x-Achse sind die Bundesländer, die vom Algorithmus geschätzt wurden, während sich auf der y-Achse die tatsächlichen Bundesländer von den Daten befinden.

**Errinnerung:** Die Bundesländer wurden als Zahlen kodiert (siehe **setup**). Burgenland: 0, Kärnten: 1, Niederösterreich: 2, Oberösterreich: 3, Salzburg: 4, Steiermark: 5, Tirol: 6, Vorarlberg: 7, Wien: 8

Vielleicht hast du bemerkt, dass die Anzahl der Daten, die hier vorkommen, weniger ist als die Größe deines Datensatzes. Das liegt daran, dass nur 30% der Daten als Testdaten verwendet werden.

In [ ]:
# Confusion Matrix
plot_model(tuned_model, plot='confusion_matrix')

### Class Prediction Error

Dies zeigt im Prinzip dieselbe Information wie unsere Confusion Matrix aber in Form eines Balkendiagrammes.

Auf der x-Achse ist das echte Bundesland und auf der y-Achse die Anzahl an Schätzungen.

**Errinnerung:** Die Bundesländer wurden als Zahlen kodiert (siehe **setup**). Burgenland: 0, Kärnten: 1, Niederösterreich: 2, Oberösterreich: 3, Salzburg: 4, Steiermark: 5, Tirol: 6, Vorarlberg: 7, Wien: 8

In [ ]:
# Fehler bei der Klassifizierung von den unterschiedlichen Bundesländern
plot_model(tuned_model, plot='error')

Es fällt auf, dass der Algorithmus oft Niederösterreich (Klasse 2) mit Oberösterreich (Klasse 3) verwechselt und umgekehrt. Wieso könnte das sein? Wir vergleichen durchschnittliche Grundstückspreise und Einwohnerzahlen von verschiedenen Bundesländern:

In [ ]:
print('Durchschnittlicher Grundstückspreis von Niederösterreich: ' + str(gem_daten_sauber[gem_daten_sauber['Bundesland'] == 'Niederösterreich']['Grundstückspreise'].mean()))
print('Durchschnittlicher Grundstückspreis von Oberösterreich: ' + str(gem_daten_sauber[gem_daten_sauber['Bundesland'] == 'Oberösterreich']['Grundstückspreise'].mean()))
print('Durchschnittlicher Grundstückspreis von Salzburg: ' + str(gem_daten_sauber[gem_daten_sauber['Bundesland'] == 'Salzburg']['Grundstückspreise'].mean()))
print('Durchschnittlicher Grundstückspreis von Kärnten: ' + str(gem_daten_sauber[gem_daten_sauber['Bundesland'] == 'Kärnten']['Grundstückspreise'].mean()))
print('Durchschnittlicher Grundstückspreis von Steiermark: ' + str(gem_daten_sauber[gem_daten_sauber['Bundesland'] == 'Steiermark']['Grundstückspreise'].mean()))

In [ ]:
print('Durchschnittliche Einwohnerzahl von Niederösterreich: ' + str(gem_daten_sauber[gem_daten_sauber['Bundesland'] == 'Niederösterreich']['Grundstückspreise'].mean()))
print('Durchschnittliche Einwohnerzahl von Oberösterreich: ' + str(gem_daten_sauber[gem_daten_sauber['Bundesland'] == 'Oberösterreich']['Grundstückspreise'].mean()))
print('Durchschnittliche Einwohnerzahl von Salzburg: ' + str(gem_daten_sauber[gem_daten_sauber['Bundesland'] == 'Salzburg']['Grundstückspreise'].mean()))
print('Durchschnittliche Einwohnerzahl von Kärnten: ' + str(gem_daten_sauber[gem_daten_sauber['Bundesland'] == 'Kärnten']['Grundstückspreise'].mean()))
print('Durchschnittliche Einwohnerzahl von Steiermark: ' + str(gem_daten_sauber[gem_daten_sauber['Bundesland'] == 'Steiermark']['Grundstückspreise'].mean()))

Wir sehen, dass Niederösterreich und Oberösterreich in einigen Spalten sehr ähnliche Werte haben. Mit der nächsten Visualisierung wirst du sehen, dass unser Algorithmus diese Werte als sehr wichtig wahrnimmt.

### Feature Importance

Beim Feature Importance Plot sieht man, welche Spalten aus unserem Gemeindedatensatz den größten Einfluss auf die Bestimmung des Bundeslandes haben. Gibt es welche, die du nicht erwartet hast?

In [ ]:
plot_model(tuned_model, plot='feature')

# Abschluss

Wenn wir mit unserem Modell zufrieden sind, wollen wir es mit **finalize_model** finalisieren. Es werden jetzt die Testdaten auch zu Trainingsdaten gemacht.


In [ ]:
final_model = finalize_model(tuned_model)

# Neue Daten vorhersagen

Wir haben uns am Anfang 2% der Daten herausgenommen, damit wir am Ende mit unserem fertigen Machine Learning Modell ein paar Schätzungen machen können. Mit **predict_model** können wir nun unser Algorithmus auf den ungesehenen Daten verwenden. 

Du wirst erkennen, dass neben 'Urbanisierungsgrad' eine neue Spalte, nämlich **Label**, dazugekommen ist. Das ist die Klassifizierung vom Algorithmus, während 'Bundesland' der echte Wert von unserer Spalte ist. 
In Zeile 1 von der Tabelle siehst du beispielsweise, dass das Bundesland von Bad Sauerbrunn eigentlich 'Burgenland' ist, aber unser Algorithmus es fälschlicherweise als 'Niederösterreich' eingeschätzt hat.

***Hinweis: Die Ausführung kann bis zu 2 Minuten dauern.***

In [ ]:
geschaetzte_daten = predict_model(final_model, data=ungesehene_daten)
geschaetzte_daten.head(42)

Falls du noch Lust hast, kannst du Machine Learning mit *Urbanisierungsgrad* (statt Bundesland) als Ziel ausprobieren! 

Was du tun musst, ist in **setup** target='Bundesland' auf target='Urbanisierungsgrad' zu ändern. Den Rest der Codezellen musst du gar nicht ändern. Wenn du magst, kannst du auch schauen, ob dort andere Algorithmen besser passen. Das bleibt alles dir überlassen!

Die Vorhersage vom Urbanisierungsgrad ist viel genauer, als die vom Bundesland. Das liegt einerseits daran, dass wir statt 9 verschiedenen Bundesländern die Urbanisierungsgrade nur in 3 Stufen einteilen.

Beim Ziel den Urbanisierungsgrad zu bestimmen erreicht Machine Learning eine Genauigkeit von fast 90%. Dies schaut zwar beeindruckend aus, aber wir stoßen auf ein Problem: Wir könnten auch schon eine Genauigkeit von 80% haben, wenn wir einen *dummen* Algorithmus bauen, der einfach allen Gemeinden den Urbanisierungsgrad 'Niedrig' zuweist.

In [ ]:
# Gemeindedaten nach Urbanisierungsgrad und Prozentanteil sortieren
gem_daten_sauber['Urbanisierungsgrad'].value_counts(normalize=True)

Hinweis:
---
Wir stoßen auf ein klassisches Problem von Machine Learning: Auch eine hohe Genauigkeit ist ohne Kontext und menschliches Hintergrundwissen nicht zwangsläufig aussagekräftig.